In [1]:
from pprint import pprint
# importing functions and modules
from main import get_options

In [2]:
debug = True

In [3]:
# defining paths to cfg 
main = 'cfg_files/main.cfg'
system = 'cfg_files/square.cfg'
saving = 'cfg_files/saving.cfg'
simulation = 'cfg_files/simulation.cfg'

cfg_path_dict = {
    'main':main,
    'system':system,
    'saving':saving,
    'simulation':simulation
}

In [4]:
options = get_options(cfg_path_dict)
if(debug): pprint(options)

{'main': {'id_test': 1,
          'particles_densities': [1e+20],
          'particles_mean_number_per_cell': [100],
          'particles_radius': [2e-10],
          'particles_types': ['I'],
          'speed_param1': [300.0],
          'speed_param2': [30.0],
          'speed_type': ['gaussian'],
          'system_type': 'square'},
 'saving': {'path': PosixPath('/home/paul/Documents/projects/Neutralisation_Faisceau_ions_negatifs_propulsion_satellite/results'),
            'period': 10},
 'simulation': {'dt': 0.25, 'number_of_steps': 300, 'scheme': 'euler_explicit'},
 'square': {'factor': 1.0, 'lz': 0.1, 'size': [3, 3]}}
